In [17]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
AZURE_EXAM_INDEX = os.getenv("AZURE_EXAM_INDEX")
AZURE_RUBRIC_INDEX = os.getenv("AZURE_RUBRIC_INDEX")

### 🗑️ Delete Index

In [15]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

index_client.delete_index(AZURE_EXAM_INDEX)
#index_client.delete_index(AZURE_RUBRIC_INDEX)
print("Old index deleted.")

Old index deleted.


### Print Index schema

In [ ]:
# Show index schema for debugging
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

index = index_client.get_index(AZURE_EXAM_INDEX)
import pprint
pprint.pprint(index.as_dict())

### Call Azure Search Index to get all documents and extract unique evidence statements

In [3]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name=AZURE_EXAM_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

# Search all documents, select only 'evidence' field
results = search_client.search("*", select=["evidence"], top=1000)

evidence_set = set()
for result in results:
    evidence = result.get('evidence', '')
    if evidence:
        for item in evidence.split("\n"):
            item = item.strip()
            if item:
                evidence_set.add(item)

for item in sorted(evidence_set):
    # Split by first '-' and trim both sides
    if '-' in item:
        key, value = item.split('-', 1)
        key = key.strip()
        value = value.strip()
    else:
        key = value = item.strip()
    print(f'"{key}" : "{value}"')

"2.a" : "Apply the legal test for consideration, including both elements of legal value and bargained-for-exchange."
"2.b" : "Understand what is meant by 'legal value' and 'bargained-for-exchange.'"
"2.c" : "Identify the legal detriment to the promisee and/or legal benefit to the promisor in a given fact pattern."
"2.d" : "Identify what is meant by the term 'consideration' in the context of contracts and gifts."
"2.e" : "Understand the concept of adequacy of consideration and the principle of 'freedom of contract.'"
"2.f" : "Understand why courts, as a general rule, do not inquire into the adequacy of consideration."


In [17]:
import pandas as pd
import json

input_path = 'output/01.02_500.jsonl'
output_path = 'output/01.02_500.csv'

records = []
with open(input_path, 'r', encoding='utf-8') as infile:
    for line in infile:
        line = line.strip()
        if not line:
            continue
        try:
            records.append(json.loads(line))
        except Exception:
            print(f"Skipping bad line: {line}")
            continue

df = pd.DataFrame(records)
if 'evidence_statements' in df.columns:
    df['evidence_statements'] = df['evidence_statements'].apply(json.dumps)
# Add empty columns for approve and reason_for_disapproval if not present
if 'approve' not in df.columns:
    df['approve'] = ''
if 'reason_for_disapproval' not in df.columns:
    df['reason_for_disapproval'] = ''
df.to_csv(output_path, index=False)

In [ ]:
%pip install python-pptx

In [12]:
from pptx import Presentation
from pathlib import Path
from typing import List, Tuple

def iter_paragraphs(shape):
    """Yield (paragraph_obj, level, text) for each paragraph in a text-containing shape."""
    if not getattr(shape, "has_text_frame", False):
        return
    for para in shape.text_frame.paragraphs:
        text = "".join(run.text for run in para.runs)
        # Keep text as-is (no strip) so spacing stays intact
        yield para, (para.level or 0), text

def group_options(paragraphs: List[Tuple[object, int, str]]) -> List[str]:
    """
    Recreate A./B./C./D. labels for consecutive 'option' paragraphs.

    Heuristic:
      - A 'stem' paragraph is any non-empty paragraph.
      - Immediately following paragraphs with the same level are treated
        as options until the level changes or a blank appears.
    """
    lines: List[str] = []
    i = 0
    while i < len(paragraphs):
        para, level, text = paragraphs[i]
        if not text.strip():
            # blank line, just keep as-is
            lines.append(text)
            i += 1
            continue

        # Emit the stem as-is
        lines.append(text)

        # Look ahead for option block
        options_start = i + 1
        option_level = None
        opts: List[str] = []

        j = options_start
        while j < len(paragraphs):
            _, lvl_j, txt_j = paragraphs[j]
            if not txt_j.strip():
                # blank breaks the option block
                break
            if option_level is None:
                option_level = lvl_j
            if lvl_j != option_level:
                # different level ends the current block
                break
            opts.append(txt_j)
            j += 1

        # If we found at least 2 option paragraphs, label them A., B., ...
        if len(opts) >= 2:
            for idx_opt, opt_text in enumerate(opts):
                label = chr(ord("A") + idx_opt) + ". "
                lines.append(label + opt_text.lstrip())
            i = j
        else:
            i += 1

    return lines

def slide_to_lines(slide) -> List[str]:
    lines: List[str] = []

    # Title
    if slide.shapes.title and slide.shapes.title.has_text_frame:
        lines.append(slide.shapes.title.text)

    # Collect all paragraphs with metadata
    paras: List[Tuple[object, int, str]] = []
    for shape in slide.shapes:
        if shape == slide.shapes.title:
            continue
        for triple in iter_paragraphs(shape):
            paras.append(triple)

    # Rebuild stems + options with labels
    lines.extend(group_options(paras))
    return lines

def pptx_to_text_with_labels(pptx_path: str, out_path: str | None = None) -> str:
    prs = Presentation(pptx_path)
    all_lines: List[str] = []

    for idx, slide in enumerate(prs.slides, start=1):
        all_lines.append(f"Slide {idx}")
        all_lines.append("-" * 40)
        all_lines.extend(slide_to_lines(slide))
        all_lines.append("")

    text = "\n".join(all_lines)
    if out_path:
        Path(out_path).write_text(text, encoding="utf-8")
    return text


if __name__ == "__main__":
    # Example usage
    input_pptx = "dataset/JD-Next Item Writing Training.pptx"
    output_txt = "dataset/JD-Next Item Writing Training.txt"
    pptx_to_text(input_pptx, output_txt)

